In [36]:
#Plan
#INPUTS: years (2x20year periods), model, experiment - these will need to be strings that match the correct filenames. Strings can also be easier then for plotting
#want to be able to change the year easily enough as then can test the sensitivity of it.
#Perhaps useful to then save the outputs (the trend map) which can then be easily used in other bits/loaded in as less computationally expensive?
#Could do similar thing with the linear trend.
#Ideally probs put this all in a function which can then take in all these inputs
#Then when changing the years easy enough to change
#Could also then loop through this function to get through all the different models and forcing scenarios.

#Import correct psl data (not CVDP but what CVDP uses) - want ens mean of hadgem3 to start (but want to maybe find a way to get all files from other models - perhaps specifying name at some point)
#calculate the ensemble mean across time? (could also only do it for the epochs but that might end up taking more time as re-running)
#get an epoch calculate the average across it, find other epoch do the same
#find the difference between the epochs
#now got the trend.
#Probs make a plot of the trend, can label with strings of model/experiment etc.


#NOTES
#could be usefult to just calculate the ensemble mean for Hadgem3 first
#This could be a good program to just run? then run it once and then it is done? yeah that would make sense, then just load in the ensemble mean data.
#


In [37]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import cftime
import pandas as pd
from datetime import datetime

In [38]:
#setting up to loop through all HadGem3 historical and for djf
def get_time_bounds(calendar_type, start, end):
    #1850-2015 all of 2014 - none of 2015.
    if calendar_type == cftime.DatetimeNoLeap:
        return cftime.DatetimeNoLeap(start,1,16), cftime.DatetimeNoLeap(end,1,16)
    elif calendar_type == cftime.Datetime360Day:
        return cftime.Datetime360Day(start,1,16), cftime.Datetime360Day(end-1,12,16)
    else:
        return datetime(start,1,16), datetime(end,1,16)

In [33]:
def calculate_spatial_ensemble_mean(file_paths, output_file, variable):
    #Will be passing through an experiment's model's ensembles.
    #opens all the files given by filepath (basically opens all the ensembles)
    ds = xr.open_mfdataset(file_paths, combine='nested', concat_dim='ensemble')

    #calculate the mean
    mean = ds[variable].mean(dim='ensemble')

    #save the ensemble mean to the a .nc file
    mean.to_netcdf(output_file)
    print('saved')

    ds.close()
    return mean

def calculate_seasonal_spatial_ensemble_mean_djf(file_path, var, seas, output_file, year_init, year_final):
    #opening dataset
    print('in function')
    ds = xr.open_dataset(file_path)
    
    #checking it is a datetime object
    ds['time'] = xr.decode_cf(ds).time

    calendar = type(ds.time.values[0])
    
    start,end = get_time_bounds(calendar, year_init, year_final)

    #selecting the psl variable within time bounds
    variable = ds[var].sel(time=slice(start, end))
    
    #Filter for the desired season (e.g., DJF)
    season_mask = variable.time.dt.season == seas
    ds_months_seas = variable.sel(time=season_mask)
    
    #assign and adjust year (DJF split over two years so increasing the year of december and then grouping and finding the mean)
    ds_months_seas = ds_months_seas.assign_coords(year=ds_months_seas['time'].dt.year)
    ds_months_seas['year'] = ds_months_seas['year'].where(ds_months_seas['time'].dt.month != 12, ds_months_seas['year'] + 1)
    #ds_months_seas = ds_months_seas.set_coords('year')
    
    # average over DJF months for each year
    ds_season = ds_months_seas.groupby('year').mean(dim='time')
    ds_season.to_netcdf(output_file)
    print('saved file')
    return ds_season

In [34]:


def get_models_for_experiment(experiment):
    if experiment == 'historical':
        model = ['ACCESS-ESM1-5','CanESM5','CMCC-CM2-SR5','FGOALS-g3','GISS-E2-1-G','HadGEM3-GC31-LL','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-LR','NorESM2-LM']
    elif experiment == 'hist-aer':
        model = ['ACCESS-ESM1-5','CanESM5','CMCC-CM2-SR5','CNRM-CM6-1','FGOALS-g3','GISS-E2-1-G','HadGEM3-GC31-LL','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-LR','NorESM2-LM']
    elif experiment == 'hist-GHG':
        model = ['ACCESS-ESM1-5','CanESM5','CMCC-CM2-SR5','CNRM-CM6-1','FGOALS-g3','GISS-E2-1-G','HadGEM3-GC31-LL','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-LR','NorESM2-LM']
    elif experiment == 'hist-sol':
        model = ['ACCESS-ESM1-5','CanESM5','GISS-E2-1-G','HadGEM3-GC31-LL','MIROC6','MPI-ESM1-2-LR','NorESM2-LM']
    elif experiment == 'hist-totalO3':
        model = ['CanESM5','GISS-E2-1-G','HadGEM3-GC31-LL','MIROC6','MPI-ESM1-2-LR','NorESM2-LM']
    elif experiment == 'hist-volc':
        model = ['ACCESS-ESM1-5','CanESM5','CMCC-CM2-SR5','GISS-E2-1-G','HadGEM3-GC31-LL','MIROC6','MPI-ESM1-2-LR','NorESM2-LM']
        
    return model

In [40]:
variable = 'psl'
experiment = ['historical', 'hist-aer','hist-GHG','hist-sol','hist-totalO3','hist-volc']

for e in experiment:
    print('Calculating the spatial ensemble means for experiment e:', e)
    model = ['HadGEM3-GC31-LL']
    for m in model:
        print('Model: ',m)
        folder_path = '/gws/nopw/j04/leader_epesc/CMIP6_SinglForcHistSimul/InterpolatedFlds/psl/'+e+'/'+m+'/'
        output_file = '/gws/nopw/j04/extant/users/slbennie/ens_mean_spat/psl/'+e+'/'+m+'/'+variable+'_mon_'+e+'_'+m+'_spatial_EM.nc'
        output_file_djf = output_file.replace('spatial_EM', 'spatial_DJF_EM_1850-2015')
        file_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if e in filename and m in filename]

        #now need to calculate the mean
        #Only need to calculate the spatial ensemble mean over all time periods can then crop
        
        #print('Calculating the spatial ensemble mean')       
        #spatial_ens_mean = calculate_spatial_ensemble_mean(file_paths, output_file, variable)

        #To use once spatial ensemble mean is calculated
        #output_file = '/gws/nopw/j04/extant/users/slbennie/ens_mean_spat/psl/historical/HadGEM3-GC31-LL/psl_mon_historical_HadGEM3-GC31-LL_spatial_EM.nc'
        print('Calculating the seasonal DJF spatial ensemble mean')
        djf_ensemble_mean = calculate_seasonal_spatial_ensemble_mean_djf(output_file, variable, 'DJF', output_file_djf, 1850,2015)
        

Calculating the spatial ensemble means for experiment e: historical
Model:  HadGEM3-GC31-LL
Calculating the seasonal DJF spatial ensemble mean
in function
saved file
Calculating the spatial ensemble means for experiment e: hist-aer
Model:  HadGEM3-GC31-LL
Calculating the seasonal DJF spatial ensemble mean
in function
saved file
Calculating the spatial ensemble means for experiment e: hist-GHG
Model:  HadGEM3-GC31-LL
Calculating the seasonal DJF spatial ensemble mean
in function
saved file
Calculating the spatial ensemble means for experiment e: hist-sol
Model:  HadGEM3-GC31-LL
Calculating the seasonal DJF spatial ensemble mean
in function
saved file
Calculating the spatial ensemble means for experiment e: hist-totalO3
Model:  HadGEM3-GC31-LL
Calculating the seasonal DJF spatial ensemble mean
in function
saved file
Calculating the spatial ensemble means for experiment e: hist-volc
Model:  HadGEM3-GC31-LL
Calculating the seasonal DJF spatial ensemble mean
in function
saved file


In [40]:
sum_ds = 0
for f in file_paths:
    ds = xr.open_dataset(f)
    sum_ds += ds['psl'].isel(lat=0,lon=0,time=0).values

print(sum_ds/len(file_paths))

print(spatial_ens_mean.isel(lat=0,lon=0,time=0).values)

seasonal = (spatial_ens_mean.isel(lat=0,lon=0,time=47).values + spatial_ens_mean.isel(lat=0,lon=0,time=48).values + spatial_ens_mean.isel(lat=0,lon=0,time=49).values)/3

print(seasonal, djf_ensemble_mean.isel(lat=0,lon=0,year=4).values)

ds_anomaly = xr.open_dataset('/gws/nopw/j04/extant/users/slbennie/psl_anomalies/historical/FGOALS-g3/psl_mon_historical_FGOALS-g3_r1i1p1f1_DJF_anomaly.nc')
ds_anomaly

99762.24103056734
99762.24103056737
99707.37871878412 <bound method Mapping.values of <xarray.Dataset> Size: 32B
Dimensions:  ()
Coordinates:
    lat      float64 8B -87.5
    lon      float64 8B -180.0
    year     int64 8B 1854
Data variables:
    psl      float64 8B 9.971e+04>


<xarray.Dataset> Size: 14MB
Dimensions:   (lat: 71, lon: 144, year: 165, bnds: 2)
Coordinates:
  * lat       (lat) float64 568B -87.5 -85.0 -82.5 -80.0 ... 80.0 82.5 85.0 87.5
  * lon       (lon) float64 1kB -180.0 -177.5 -175.0 ... 172.5 175.0 177.5
  * year      (year) int64 1kB 1850 1851 1852 1853 1854 ... 2011 2012 2013 2014
    season    <U3 12B ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds  (year, lat, bnds) float64 187kB ...
    lon_bnds  (year, lon, bnds) float64 380kB ...
    psl       (year, lat, lon) float64 13MB ...